In [ ]:
%matplotlib qt
import mne

In [ ]:
%matplotlib qt
from eeg_preprocessing import preprocessing
preprocessing(subject_id="zdfy",
                subjects_dir="/Users/payamsadeghishabestari/antinomics_clean_codes/subjects",
                paradigm="gpias",
                manual_data_scroll=True,
                run_ica=False,
                manual_ica_removal=False,
                eog_correct=True,
                pulse_correct=True,
                resp_correct=False,
                create_report=True,
                saving_dir=None,
                verbose="ERROR")

In [ ]:
%matplotlib qt
from eeg_processing import run_rs_analysis, run_erp_analysis
run_erp_analysis(
        subject_id="zdfy",
        subjects_dir="/Users/payamsadeghishabestari/antinomics_clean_codes/subjects",
        paradigm="gpias",
        events=None,
        source_analysis=True,
        mri=False,
        subjects_fs_dir=None,
        manual_data_scroll=False,
        create_report=True,
        saving_dir=None,
        verbose="ERROR"
        )

Eyetracking

In [ ]:
import mne
raw = mne.io.read_raw_brainvision("/Users/payamsadeghishabestari/antinomics_clean_codes/subjects/zdfy/EEG/xxxxx/zdfy_xxxxx.vhdr")
mne.read_events(raw)

In [ ]:
raw.annotations

In [ ]:
from mne.io import read_raw_brainvision, read_raw_eyelink
from mne import events_from_annotations
from mne.preprocessing import realign_raw


## load
fname_eeg = "/Users/payamsadeghishabestari/Downloads/alertness_01_H036.vhdr"
fname_et = "/Users/payamsadeghishabestari/Downloads/local_al1H036.asc"

raw_eeg = read_raw_brainvision(fname_eeg, preload=True)
raw_eye = read_raw_eyelink(fname=fname_et, create_annotations=True) 
events_eeg, event_ids_eeg = events_from_annotations(raw_eeg)
events_eye, event_ids_eye = events_from_annotations(raw_eye)

## for now
stim_id_eeg_1, stim_id_eeg_2 = 8, 9
stim_id_eye_1, stim_id_eye_2 = 4, 5

s_raw = events_eeg[(events_eeg[:,2] == stim_id_eeg_1) | (events_eeg[:,2] == stim_id_eeg_2)][:, 0]
s_other = events_eye[(events_eye[:,2] == stim_id_eye_1) | (events_eye[:,2] == stim_id_eye_2)][:, 0]

## realigning
realign_raw(raw=raw_eeg,
            other=raw_eye,
            t_raw=s_raw / raw_eeg.info["sfreq"] - raw_eeg.first_time,
            t_other=s_other / raw_eye.info["sfreq"] - raw_eye.first_time,
            verbose=None)

raw_eye.add_channels([raw_eeg], force_update_info=True)
events, event_dict = events_from_annotations(raw_eye)
del raw_eeg  

In [ ]:
fname_eeg = "/Users/payamsadeghishabestari/Downloads/alertness_01_H036.vhdr"
fname_et = "/Users/payamsadeghishabestari/Downloads/local_al1H036.asc"

raw_eeg = read_raw_brainvision(fname_eeg, preload=True)
raw_eye = read_raw_eyelink(fname=fname_et, create_annotations=True) 

Lets see what is regularity

In [1]:
import mne

In [3]:
fname = "/Users/payamsadeghishabestari/antinomics_clean_codes/subjects/dvob/EEG/regularity/dvob_regularity.vhdr"
raw = mne.io.read_raw_brainvision(fname, preload=True)
raw.resample(250)

Extracting parameters from /Users/payamsadeghishabestari/antinomics_clean_codes/subjects/dvob/EEG/regularity/dvob_regularity.vhdr...
Setting channel info structure...
Reading 0 ... 2210759  =      0.000 ...  2210.759 secs...


/var/folders/20/hsy69tx529ndn3rkv5gzcf0c0000gn/T/ipykernel_2072/2644501620.py:2: RuntimeWarning: No coordinate information found for channels ['X', 'Y', 'Z', 'Photo', 'Audio']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(fname, preload=True)
/var/folders/20/hsy69tx529ndn3rkv5gzcf0c0000gn/T/ipykernel_2072/2644501620.py:2: RuntimeWarning: Not setting positions of 8 misc channels found in montage:
['Pulse', 'GSR', 'X', 'Y', 'Z', 'Resp', 'Photo', 'Audio']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_brainvision(fname, preload=True)


<RawBrainVision | dvob_regularity.eeg, 71 x 552690 (2210.8 s), ~299.5 MB, data loaded>

In [7]:
events, events_dict = mne.events_from_annotations(raw)

Used Annotations descriptions: ['New Segment/', 'Stimulus/S  1', 'Stimulus/S  2', 'Stimulus/S  3', 'Stimulus/S  4', 'Stimulus/S  5', 'Stimulus/S  6', 'Stimulus/S  7', 'Stimulus/S  8', 'Stimulus/S 11', 'Stimulus/S 12', 'Stimulus/S 13', 'Stimulus/S 14', 'Stimulus/S 15', 'Stimulus/S 16', 'Stimulus/S 17', 'Stimulus/S 18', 'Stimulus/S140']
